### Домашнее задание

1. обучить несколько разных моделей на наборе данных ССЗ (train_case2.csv): логрег, бустинг, лес и т.д - на ваш выбор 2-3 варианта
2. при обучении моделей обязательно использовать кроссвалидацию
3. вывести сравнение полученных моделей по основным метрикам классификации: pr/rec/auc/f_score (можно в виде таблицы, где строки - модели, а столбцы - метрики)
4. сделать выводы о том, какая модель справилась с задачей лучше других
5. (опциональный вопрос) какая метрика (precision_recall_curve или roc_auc_curve) больше подходит в случае сильного дисбаланса классов? (когда объектов одного из классов намного больше чем другого).

p.s.В вопросе проще разобраться, если вспомнить оси на графике roc auc curve и рассмотреть такой пример:

Имеется 100000 объектов, из которых только 100 - класс "1" (99900 - класс "0", соответственно).
Допустим, у нас две модели:

- первая помечает 100 объектов как класс 1, но TP = 90
- вторая помечает 1000 объектов как класс 1, но TP такой же - 90

Какая модель лучше и почему? И что позволяет легче сделать вывод - roc_auc_curve или precision_recall_curve?

In [111]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, roc_auc_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier as xgbc
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
# import warnings
# warnings.filterwarnings('ignore')

In [112]:
df = pd.read_csv('train_case2.csv', ';')
df.head(3)

,id,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,2,18857,1,165,64.0,130,70,3,1,0,0,0,1


Разделим наши данные на тренировочную и тестовую выборки

In [113]:
#разделим данные на train/test
X_train, X_test, y_train, y_test = train_test_split(df.drop('cardio', 1),
                                                    df['cardio'], random_state=0)

К полям:
- gender, cholesterol применим OHE-кодирование
- age, height, weight, ap_hi, ap_lo - standardScaler
- gluc, smoke, alco, active - оставим пока как есть

In [114]:
class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]

class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]

class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]


from sklearn.preprocessing import StandardScaler


continuos_cols = ['age', 'height', 'weight', 'ap_hi', 'ap_lo']
cat_cols = ['gender', 'cholesterol']
base_cols = ['gluc', 'smoke', 'alco', 'active']

continuos_transformers = []
cat_transformers = []
base_transformers = []

for cont_col in continuos_cols:
    transfomer =  Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('standard', StandardScaler())
            ])
    continuos_transformers.append((cont_col, transfomer))

for cat_col in cat_cols:
    cat_transformer = Pipeline([
                ('selector', ColumnSelector(key=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    cat_transformers.append((cat_col, cat_transformer))

for base_col in base_cols:
    base_transformer = Pipeline([
                ('selector', NumberSelector(key=base_col))
            ])
    base_transformers.append((base_col, base_transformer))

Теперь объединим все наши трансформеры с помощью FeatureUnion

In [115]:
from sklearn.pipeline import FeatureUnion

feats = FeatureUnion(continuos_transformers+cat_transformers+base_transformers)
feature_processing = Pipeline([('feats', feats)])

feature_processing.fit_transform(X_train)

array([[-1.73391771,  0.6873301 ,  0.74843904, ...,  1.        ,
         0.        ,  1.        ],
       [-1.67343538,  0.07758923, -0.29640123, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.13738132,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 1.17775864,  1.17512278, -0.15708919, ...,  0.        ,
         0.        ,  1.        ],
       [-0.47190715, -1.38578883,  0.74843904, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.38174619,  0.56538192, -0.08743318, ...,  0.        ,
         0.        ,  1.        ]])

In [116]:
def model(model):
    classifier = Pipeline([
        ('features',feats),
        ('classifier', model),
    ])


    #запустим кросс-валидацию
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=7, scoring='roc_auc')
    cv_score = np.mean(cv_scores)
    cv_score_std = np.std(cv_scores)
    print('CV score is {}+-{}'.format(cv_score, cv_score_std))

    #обучим пайплайн на всем тренировочном датасете
    classifier.fit(X_train, y_train)
    y_score = classifier.predict_proba(X_test)[:, 1]
    return y_score, classifier

In [117]:
def metrics(y_score, classifier):
    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix],
                                                                            fscore[ix],
                                                                            precision[ix],
                                                                            recall[ix]))
    roc_auc = roc_auc_score(y_true=y_test, y_score=classifier.predict_proba(X_test)[:,1])

    cnf_matrix = confusion_matrix(y_test, y_score>thresholds[ix])
    TN = cnf_matrix[0][0]
    FN = cnf_matrix[1][0]
    TP = cnf_matrix[1][1]
    FP = cnf_matrix[0][1]

    TPR = TP/(TP+FN)
    FPR = FP/(FP+TN)
    TNR = TN/(FP+TN)



    m_values = [precision[ix],
                recall[ix],
                fscore[ix],
                roc_auc,
                TPR,
                FPR,
                TNR]

    return m_values

In [118]:
# Создадим таблицу для сравнения метрик
results = pd.DataFrame(columns=['Model','Precision', 'Recall', "Roc Auc", 'F-Score', 'TPR', 'FPR', 'TNR'])
results['Model'] = ['LogReg','XGBoost', 'Random Forest Classifier']
results.set_index('Model', drop=True, inplace=True)

In [119]:
# Первая модель логистическая регрессия
y_score, classifier = model(LogisticRegression(random_state = 42))

CV score is 0.7864573689384385+-0.004422021036885763


In [120]:
# Считаем метрики
metrics_values = metrics(y_score, classifier)

Best Threshold=0.386937, F-Score=0.730, Precision=0.647, Recall=0.838


In [121]:
# Добавляем в таблицу
results.loc['LogReg'] = metrics_values

In [122]:
# Вторая модель XGBoost
y_score, classifier = model(xgbc(random_state = 42, verbosity = 0))

CV score is 0.7971556462159913+-0.0025807632015828223


In [123]:
# Считаем метрики
metrics_values = metrics(y_score, classifier)

Best Threshold=0.347103, F-Score=0.738, Precision=0.665, Recall=0.828


In [124]:
# Добавляем в таблицу
results.loc['XGBoost'] = metrics_values

In [125]:
# Третья модель Random Forest Classifier
y_score, classifier = model(rfc(random_state = 42))

CV score is 0.7743796633622809+-0.003611330100912611


In [126]:
# Считаем метрики
metrics_values = metrics(y_score, classifier)

Best Threshold=0.350000, F-Score=0.719, Precision=0.643, Recall=0.816


In [127]:
# Добавляем в таблицу
results.loc['Random Forest Classifier'] = metrics_values

In [128]:
results

,Precision,Recall,Roc Auc,F-Score,TPR,FPR,TNR
Model,,,,,,,
LogReg,0.647431,0.837558,0.730323,0.784035,0.837442,0.448866,0.551134
XGBoost,0.665248,0.828341,0.73789,0.797228,0.828226,0.410204,0.589796
Random Forest Classifier,0.642669,0.815553,0.718863,0.771037,0.80841,0.434127,0.565873


Все модели показали примерно одинаковые результаты, хотя по многим метрикам XGBoost немного лучше. Для данного типа задач имеет смысл максимизировать precision, так как он помогает максимально предсказывать Positive объекты, что важно в вопросах медицинской диагностики.
